In [12]:
# For this assignment, I will be taking this website (Top 10 Amusement Parks), 
# and extracting data to create both a point file that contains the locations of these food trucks throughout the week

# ------------------------------------------------------------------------------------------------------ #
# SETUP FOR ALL SECTIONS #
import urllib2
import lxml.html
import os
import sys
import ggeocoder
import csv
from subprocess import call
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\ArcToolbox\\Scripts')
import arcpy
from arcpy import env
env.overwriteOutput = True
env.workspace = r"C:\Users\osterj\Desktop\temp"
os.environ["GDAL_DATA"] = "C:\\OSGeo4W\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:\\OSGeo4W\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W\\share\\proj"
os.environ["PATH"] = "C:\\OSGeo4W\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W\\apps\\msys\\bin;C:\\OSGeo4W\\apps\\Python27\\Scripts"

# ------------------------------------------------------------------------------------------------------ #

def page_getter(url):
    html = urllib2.urlopen(url).read()
    dom = lxml.html.fromstring(html)
    dom.make_links_absolute(url)
    return dom

website = r"http://www.travelchannel.com/interests/family/articles/top-10-amusement-parks"

# ------------------------------------------------------------------------------------------------------ #

dom = page_getter(website) 
parks = dom.cssselect(".poi-title a")     
park_url_list = [link.attrib['href'] for link in parks]
pnl = [link.text for link in parks]
park_name_list = []

# park name list is returning "\nPARKNAME" so I need to delete the \n
for name in pnl:
    park_name_list.append(name.splitlines()[1])
# splitlines is a list that = [' ', 'PARKNAME'], so splitlines()[1] takes index 1 of the splitlines list, aka PARKNAME. 
    
# makes a list of the park names with underscores where there are spaces
underscore = []    
for name in park_name_list:
    new_name = ""
    for char in name:        
        if char == ' ' or char == '\'':
            char = '_'
        new_name += char
    underscore.append(new_name)    
# print underscore

# make a list of the full city addresses
streets = dom.cssselect(".street-address")
streets_list = [addr.text for addr in streets]
# print streets_list

# ------------------------------------------------------------------------------------------------------ #

# This next Section I'll be breaking up the full city address into its individual components
# make a list of the cities
cities_full = dom.cssselect(".city")
city_list_full = [city.text for city in cities_full]
# print city_list_full

# cities_full contains city name COMMA state code SPACE zip code SPACE US, 
# so I should be able to take each string working backwards to make lists of 
# the country, then zip, then state, then city name
# hopefully this for loop can make all 4 lists in 1 go
cities = []
states = []
zips = []
countries = []
for city in city_list_full:
    citye = ""
    statee = ""
    zipe = ""
    countrye = ""
    currLen = 0
    len_city = len(city)
    # example: SU 07844 HO ,yksudnaS
    # example: countrye, zipe, statee, citye
    for char in reversed(city):
        if currLen < 2:
            countrye = char + countrye
            currLen += 1
        elif currLen == 2:
            currLen += 1
        elif currLen < 8:
            zipe = char + zipe
            currLen += 1
        elif currLen == 8:
            currLen += 1
        elif currLen < 11:
            statee = char + statee
            currLen += 1
        elif currLen == 11 or currLen == 12:
            currLen += 1
        elif currLen > 12 and currLen < len_city:
            citye = char + citye
            currLen += 1
        else:
            pass
    countries.append(countrye)
    zips.append(zipe)
    states.append(statee)
    cities.append(citye)

# make list of full addresses
full_add = []
strt = 0
for park in park_name_list:
    full_add.append(streets_list[strt] + ", " + 
                    cities[strt] + ", " +
                    states[strt] + ", " + 
                    zips[strt] + ", " + 
                    countries[strt])
    strt += 1

# print countries
# print zips
# print states
# print cities
# print full_add

# ------------------------------------------------------------------------------------------------------ #

# To clarify my code at this point, this is a list of all the lists I have at this point:

# underscore       = park names with underscores where there are "'"s and " "s
# park_url_list    = park URLs
# pnl              = park names with "\n" at the beginning of each park name
# park_name_list   = park names "properly" written out
# streets_list     = park street addresses
# cities           = city the park is located in 
# states           = state the park is located in
# zips             = zip code of the park
# countries        = country of the park (they are all US in this case)
# city_list_full   = park city, state, zip, and country combined into 1 line (ex. Sandusky, OH 44870 US )
# full_add         = full address of the park, delineated with commas

# ------------------------------------------------------------------------------------------------------ #

# for park_info in underscore:
#     print park_info
    
# ------------------------------------------------------------------------------------------------------ #  

# This section will be about getting these dictionaries and geocoding them
new_full_add = []
coordinates = []
g = ggeocoder.Geocoder()
for add in full_add:
    results = g.geocode(add)
    i = results[0]
    new_full_add.append(i.formatted_address)
    coordinates.append(i.coordinates)
    
# from the coordinates, separate latitudes from longitudes and put them into their own lists
latitudes = []
longitudes = []
for c in coordinates:
    latitudes.append(c[0])
    longitudes.append(c[1])

# print latitudes
# print longitudes
    
# print new_full_add
# print coordinates

# make a dictionary named each theme park name, and put in the name, street address, state, zip, country
# and the full addresses as found by the geocoder as well as the coordinates
start = 0
for name in underscore:
    underscore[start] = {"Name" : park_name_list[start], 
                         "Address" : streets_list[start], 
                         "City" : cities[start], 
                         "State" : states[start], 
                         "Zip Code" : zips[start],
                         "Country" : countries[start],
                         "URL" : park_url_list[start],
                         "Full Address" : new_full_add[start],
                         "Latitude" : latitudes[start],
                         "Longitude" : longitudes[start]}
    # print str(underscore[start])
    start += 1  

# print underscore        
# Underscore is now a list of dictionaries containing all the relevant information

# ------------------------------------------------------------------------------------------------------ #  

# converting this "lovely" list of dictionaries into a CSV file

field_names = ['City', 'State', 'Full Address', 'Name', 'Address', 'URL', 'Country', 'Zip Code', 'Latitude', 'Longitude']
test_file = open(r"C:\Users\osterj\Desktop\temp\themeparks.csv",'wb')
csvwriter = csv.DictWriter(test_file, delimiter = ',', fieldnames = field_names)
csvwriter.writerow(dict((fn,fn) for fn in field_names))
for row in underscore:
     csvwriter.writerow(row)
test_file.close()

# ------------------------------------------------------------------------------------------------------ # 

# make the shape file by making an xy event layer and copying it over to a .shp
# GCS_WGS_1984 is the projection of this shp file, the datum is D_WGS_1984
xy_layer = arcpy.MakeXYEventLayer_management(r"C:\Users\osterj\Desktop\temp\themeparks.csv", "Longitude", "Latitude", "theme_parks")
arcpy.CopyFeatures_management(xy_layer, r"C:\Users\osterj\Desktop\temp\themeparks.shp")

call(['C:\\OSGeo4W\\bin\\ogr2ogr',
      '-f','GeoJSON','-t_srs','WGS84',
      '-s_srs','WGS84',
      r"C:\Users\osterj\Desktop\temp\themeparks.geojson",
      r"C:\Users\osterj\Desktop\temp\themeparks.shp"])

0

# README NOTES:
## Requirements:
1. For people who don't go go theme parks all that often, I suppose this project would be fairly intellectually significant. Otherwise I'm not really sure how this project was intellectually significant other than for me to learn how to scrap data off of a website and turn it into spatial data. 

2. There were definitely plenty of challenges in this project, from trying to figure out how to select loose text within an html tag (the a-tag for this particular project) was one problem I ran into at the very beginning. The mid section of this project went a lot smoother, as it was mostly just taking the data I had already pulled down through the scraper and converting it into a multiple of lists. One interesting solution I found involved the city address line, as it contained the city name, the state, zip code, and country of the theme park, but it didn't delineate them the same way and it was in an odd order. There was no decent way to separate the data; however since the codes for country, zip, and state were all of the same length, I decided to have the code read that line backwards (to account for the varying length of the city names) and reconstruct each segment by appending character by character to the front of a new string from which I could append into lists. 

3. The biggest challenge was figuring out how to convert my list of dictionaries into a CSV, and then from a CSV to a SHP. This is where William Chan helped me out immensely, as he was a bit ahead of me in the process, he had found some pitfalls in the process and helped guide me through the steps I needed to take. Luckily though I remembered the ogr2ogr function from the first lab in this class so it was fairly easy to go back and readjust to fit this project. And luckily the .shp file creation in this project was already in the WGS84 projection so I didn't have to try and figure out what project to convert from.

4. I managed to put this entire code together from beginning to end on 02/17/2016, I believe in total it took me about 6 hours, give or take.